# Sensitivities and symbolic differentiation

The standard phrasing of forward sensitivities is

 - State: $y = y(p, t)$
 - Constant or an initial state value: $p$ 
 - State derivatives: $f(y, p) = \frac{\mathrm{d}y}{\mathrm{d}t}$, this is the myokit.Model
 - Sensitivities: $s = \frac{\mathrm{d}y}{\mathrm{d}p}$
 - Sensitivity ODEs: $\frac{\mathrm{d}s}{\mathrm{d}t} = \frac{\partial f}{\partial y}s + \frac{\partial f}{\partial p}$

The final sensitivity ODE expression assumes that there are no intermediate expressions that may implicitly depend on time, parameters or other states. So when computing the sensitivities, it is important to plug in all intermediate expressions into $f$.

In myokit, we might also be interested in the sensitivities of an intermediate expression. Those can be stratghtforwadly derived from the above sensitivities

 - Intermediate: $x = x(y, p, t)$
 - Sensitivities: $\frac{\mathrm{d}x}{\mathrm{d}p} = \frac{\partial x}{\partial y}s + \frac{\partial x}{\partial p}$.

Here, again, we'll assume that any dependencies of x on other intermediate expressions are plugged in, prior to the computation of the derivatives.

In [3]:
import myokit

## Model

(For simplicity units of constants are dropped)

State equations:
\begin{eqnarray}
    \frac{\mathrm{d}{Ca}_i}{\mathrm{d}t} &=& -0.001 ICa + 0.07 (0.0001 - Ca_i)\\
    \frac{\mathrm{d}d}{\mathrm{d}t} &=& \alpha_d (1 - d) - \beta_d d \\
    \frac{\mathrm{d}f}{\mathrm{d}t} &=& \alpha_f (1 - f) - \beta_f f \\
    \frac{\mathrm{d}x}{\mathrm{d}t} &=& \alpha_x (1 - x) - \beta_x x \\
    \frac{\mathrm{d}h}{\mathrm{d}t} &=& \alpha_h (1 - h) - \beta_h h \\
    \frac{\mathrm{d}m}{\mathrm{d}t} &=& \alpha_m (1 - m) - \beta_m m \\
    \frac{\mathrm{d}j}{\mathrm{d}t} &=& \alpha_j (1 - j) - \beta_j j \\
    \frac{\mathrm{d}V}{\mathrm{d}t} &=& -(i_{ion} + i_{diff} + i_{stim}) / C
\end{eqnarray}

Algebraic equations:
\begin{eqnarray}
    Ib &=& gb (V - Eb) \\
    E_{ica} &=& 7.7 - 13.0287 \log (Ca_i / Ca_o) \\
    ICa &=& gCa\, d \, f (V-E_{ica}) \\
    \alpha _d &=& 0.095 \frac{\exp(-0.01 (V - 5))}{1 + \exp(-0.072 (V - 5))} \\
    \beta _d &=& 0.07 \frac{\exp(-0.017 (V + 44))}{1 + \exp(0.05 (V + 44))} \\
    \alpha _f &=& 0.012 \frac{\exp(-0.008 (V + 28))}{1 + \exp(0.15 (V + 28))} \\
    \beta _f &=& 0.0065 \frac{\exp(-0.02 (V + 30 [mV]))}{1 + \exp(-0.2(V + 30))} \\
    IK &=& gK \, xi \, x \, (V-E_{ik}) \\
    E_{ik} &=& RTF \log \frac{K_o \, PNa_K\, Na_o}{K_i\, PNa_K\, Na_i} \\
    gK &=& 0.282 \sqrt{K_o / 5.4} \\
    \alpha _x &=& 0.0005\frac{\exp(0.083 (V + 50))}{1 + \exp(0.057 (V + 50))} \\
    \beta _x &=& 0.0013\frac{\exp(-0.06 (V + 20 ))}{1 + \exp(-0.04(V + 20))} \\
    xi &=& 
        \left\lbrace
        \begin{matrix}
             1 & \mathrm{for}\quad V<-100\\ 
             2.837 \frac{0.04}{\exp(0.04(V + 35)} & \mathrm{for}\quad V=-77 \\
             2.837 \frac{\exp(0.04 (V + 77 )) - 1}{(V + 77) \exp(0.04 (V + 35))} & \mathrm{else}

        \end{matrix}
        \right. \\
    E_{ik1} &=& RTF\log(K_o / K_i) \\
    IK1 &=& gK1\, g \, (V-E_{ik1}) \\
    g &=& \frac{\alpha _g}{\alpha _g + \beta _g} \\
    \alpha _g &=& \frac{1.02}{1 + \exp(0.2385 (V - E_{ik1} - 59.215))} \\
    \beta _g &=& \frac{0.49124 \exp(0.08032(V - E_{ik1} + 5.476)) + \exp(0.06175 (V - E_{ik1} - 594.31))}{1 + \exp(-0.5143 (V - E_{ik1} + 4.753))} \\
    gK1 &=& 0.6047\sqrt{K_o / 5.4} \\
    IKp &=& gKp \, Kp (V - E_{ik1})\\
    Kp &=& \frac{1}{5.98(1 + \exp 7.448 - V)} \\
    ENa &=& RTF \log (Na_o / Na_i) \\
    INa &=& gNa \, m^3\, h\, j (V-ENa) \\
    a &=& 1 - \frac{1}{1 + \exp (-(V + 40)) / 0.24} \\
    \alpha _h &=& a 0.135 \exp(-(80+ V) / 6.8 ) \\
    \beta _h &=& a (3.56 \exp(0.079 V) + 310000  \exp(0.35 V)) + \frac{1 - a}{0.13 (1 + \exp(-(V + 10.66) / 11.1))} \\
    \alpha _j &=& a (-127140 \exp(0.2444 V) - 3.474e-5 \exp(-0.04391 V))\frac{V + 37.78}{1 + \exp(0.311(V + 79.23))}\\
    \beta_j &=& a \frac{0.1212 \exp(-0.01052 V)}{1 + \exp(-0.1378(V + 40.14 [mV]))} + (1 - a) \frac{0.3 \exp(-2.535e-7 V)}{1 + \exp(-0.1 (V + 32))} \\
    \alpha _m &=& 0.32 \frac{V + 47.13}{1 - \exp(-0.1(V + 47.13))} \\
    \beta _m &=& 0.08 \exp(-V / 11) \\
    i_{ion} &=& INa + IK + Ib + IKp + IK1 + ICa\\
    i_{stim} &=& \mathrm{pace} * \mathrm{stimulous}
\end{eqnarray}

Constants:
\begin{eqnarray}
    Ca_o &=& 1.8\\
    K_i &=& 145\\
    K_o &=& 5.4\\
    Na_i &=& 10\\
    Na_o &=& 140\\
    RTF &=& 8314 * 310 / 96500\\
    Eb &=& -59.87\\
    gb &=& 0.03921\\
    gCa &=& 0.09 \\
    PNa_K &=& 0.01833 \\
    gKp &=& 0.0183 \\
    gNa &=& 16 \\
    C &=& 1 \\
    i_{diff} &=& 0 \\
    \mathrm{stimulous} &=& -80 
\end{eqnarray}

In [4]:
model = myokit.load_model('example')

print(model.code())

[[model]]
desc: """
    Implementation of the Luo-Rudy model I for the ventricular myocyte.
    Based on an updated formulation downloaded from http://rudylab.wustl.edu
    and licensed under a GNU GPL license version 2.
    """
name: Luo-Rudy 1991
# Initial values
membrane.V = -84.5286
ina.m      = 0.0017
ina.h      = 0.9832
ina.j      = 0.995484
ica.d      = 3e-6
ica.f      = 1
ik.x       = 0.0057
ica.Ca_i   = 0.0002

[cell]
desc: Contains a list of cell constants
Ca_o = 1.8 [mM]
    in [mM]
K_i = 145 [mM]
    in [mM]
K_o = 5.4 [mM]
    in [mM]
Na_i = 10 [mM]
    in [mM]
Na_o = 140 [mM]
    in [mM]
RTF = R * T / F
    in [mV]
    F = 96500 [C/mol] : Faraday constant (Value used in LR model)
        in [C/mol]
    R = 8314 [mJ/mol/K] : Gas constant
        in [mJ/mol/K]
    T = 310 [K] : Absolute temperature
        in [K]

[engine]
desc: Simulation engine variables
pace = 0 bind pace
time = 0 [ms] bind time
    in [ms]

[ib]
desc: Background current
Eb = -59.87 [mV]
    in [mV]
Ib = 

## Compute sensitivities

### Example I: Sensitivities of $Ca_i$
The calcium count may be measured in experiments and thus may be a model output with sensitivities of interest.

#### Analytic sensitivity ODE
The relevant expressions from the above model are

\begin{eqnarray}
    \frac{\mathrm{d}{Ca}_i}{\mathrm{d}t} &=& -0.001 ICa + 0.07 (0.0001 - Ca_i)\\
    ICa &=& gCa\, d \, f (V-E_{ica}) \\
    gCa &=& 0.09 \\
    \frac{\mathrm{d}d}{\mathrm{d}t} &=& \alpha_d (1 - d) - \beta_d d \\
    \frac{\mathrm{d}f}{\mathrm{d}t} &=& \alpha_f (1 - f) - \beta_f f \\
    \frac{\mathrm{d}V}{\mathrm{d}t} &=& -(i_{ion} + i_{diff} + i_{stim}) / C \\
    E_{ica} &=& 7.7 - 13.0287 \log (Ca_i / Ca_o) \\
    \alpha _d &=& 0.095 \frac{\exp(-0.01 (V - 5))}{1 + \exp(-0.072 (V - 5))} \\
    \beta _d &=& 0.07 \frac{\exp(-0.017 (V + 44))}{1 + \exp(0.05 (V + 44))} \\
    \alpha _f &=& 0.012 \frac{\exp(-0.008 (V + 28))}{1 + \exp(0.15 (V + 28))} \\
    \beta _f &=& 0.0065 \frac{\exp(-0.02 (V + 30))}{1 + \exp(-0.2(V + 30))} \\
    i_{ion} &=& INa + IK + Ib + IKp + IK1 + ICa\\
    i_{diff} &=& 0 \\
    i_{stim} &=& \mathrm{pace} * \mathrm{stimulous} \\
    C &=& 1 \\
    Ca_o &=& 1.8\\
    INa &=& gNa \, m^3\, h\, j (V-ENa) \\
    IK &=& gK \, xi \, x \, (V-E_{ik}) \\
    Ib &=& gb (V - Eb) \\
    IKp &=& gKp \, Kp (V - E_{ik1})\\
    IK1 &=& gK1\, g \, (V-E_{ik1}) \\
    \mathrm{stimulous} &=& -80 \\
    gNa &=& 16 \\
    \frac{\mathrm{d}m}{\mathrm{d}t} &=& \alpha_m (1 - m) - \beta_m m \\
    \frac{\mathrm{d}h}{\mathrm{d}t} &=& \alpha_h (1 - h) - \beta_h h \\
    \frac{\mathrm{d}j}{\mathrm{d}t} &=& \alpha_j (1 - j) - \beta_j j \\
    ENa &=& RTF \log (Na_o / Na_i) \\
    gK &=& 0.282 \sqrt{K_o / 5.4} \\
    xi &=& 
        \left\lbrace
        \begin{matrix}
             1 & \mathrm{for}\quad V<-100\\ 
             2.837 \frac{0.04}{\exp(0.04(V + 35)} & \mathrm{for}\quad V=-77 \\
             2.837 \frac{\exp(0.04 (V + 77 )) - 1}{(V + 77) \exp(0.04 (V + 35))} & \mathrm{else}

        \end{matrix}
        \right. \\
    \frac{\mathrm{d}x}{\mathrm{d}t} &=& \alpha_x (1 - x) - \beta_x x \\
    E_{ik} &=& RTF \log \frac{K_o \, PNa_K\, Na_o}{K_i\, PNa_K\, Na_i} \\
    gb &=& 0.03921\\
    Eb &=& -59.87\\
    gKp &=& 0.0183 \\
    Kp &=& \frac{1}{5.98(1 + \exp 7.448 - V)} \\
    E_{ik1} &=& RTF\log(K_o / K_i) \\
    gK1 &=& 0.6047\sqrt{K_o / 5.4} \\
    g &=& \frac{\alpha _g}{\alpha _g + \beta _g} \\
     &.& \\
    &.& \\
    &.&
\end{eqnarray}

Not feasible to show sensitivity when voltage is coupling the $Ca_i$ count to all other currents in the model. In other words, $Ca_i$ is sensitive to all parameters in the model.

### Example II: Sensitivities of Ca when voltage is constant and externally controlled

#### Analytic sensitivity ODE
Now, the relevant expressions from the above model are

\begin{eqnarray}
    \frac{\mathrm{d}{Ca}_i}{\mathrm{d}t} &=& -0.001 ICa + 0.07 (0.0001 - Ca_i)\\
    ICa &=& gCa\, d \, f (V-E_{ica}) \\
    gCa &=& 0.09 \\
    \frac{\mathrm{d}d}{\mathrm{d}t} &=& \alpha_d (1 - d) - \beta_d d \\
    \frac{\mathrm{d}f}{\mathrm{d}t} &=& \alpha_f (1 - f) - \beta_f f \\
    V &=& 1 \\
    E_{ica} &=& 7.7 - 13.0287 \log (Ca_i / Ca_o) \\
    \alpha _d &=& 0.095 \frac{\exp(-0.01 (V - 5))}{1 + \exp(-0.072 (V - 5))} \\
    \beta _d &=& 0.07 \frac{\exp(-0.017 (V + 44))}{1 + \exp(0.05 (V + 44))} \\
    \alpha _f &=& 0.012 \frac{\exp(-0.008 (V + 28))}{1 + \exp(0.15 (V + 28))} \\
    \beta _f &=& 0.0065 \frac{\exp(-0.02 (V + 30))}{1 + \exp(-0.2(V + 30))} \\
    Ca_o &=& 1.8
\end{eqnarray}

In order to derive the sensitivities according to $\frac{\mathrm{d}s}{\mathrm{d}t} = \frac{\partial f}{\partial y}s + \frac{\partial f}{\partial p}$, we have to make sure that the rhs of all states only depends on states or parameters:

\begin{eqnarray}
    \frac{\mathrm{d}{Ca}_i}{\mathrm{d}t} &=& -0.001 gCa\, d \, f (V-(7.7 - 13.0287 \log (Ca_i / Ca_o))) + 0.07 (0.0001 - Ca_i)\\
    \frac{\mathrm{d}d}{\mathrm{d}t} &=& 0.095 \frac{\exp(-0.01 (V - 5))}{1 + \exp(-0.072 (V - 5))} (1 - d) - 0.07 \frac{\exp(-0.017 (V + 44))}{1 + \exp(0.05 (V + 44))} d \\
    \frac{\mathrm{d}f}{\mathrm{d}t} &=& 0.012 \frac{\exp(-0.008 (V + 28))}{1 + \exp(0.15 (V + 28))} (1 - f) - 0.0065 \frac{\exp(-0.02 (V + 30))}{1 + \exp(-0.2(V + 30))} f \\
    gCa &=& 0.09 \\
    V &=& 1 \\
    Ca_o &=& 1.8
\end{eqnarray}

Thus, $Ca_i$ is sensitive to three parameters $(gCa, V, Ca_o)$ as well as the initial conditions of the states.

1. Sensitivity to $gCa$: 
    The states $d$ and $f$ do not depend on $gCa$, thus their sensitivities are zero. As a result, the only contribution comes from the rhs of $Ca_i$:
    \begin{eqnarray}
        \frac{\mathrm{d}}{\mathrm{d}t}\frac{\mathrm{d}{Ca}_i}{\mathrm{d}gCa} &=& \left( -0.0130287 gCa\, d \, f / Ca_i - 0.07 \right) \frac{\mathrm{d}{Ca}_i}{\mathrm{d}gCa} -0.001 d \, f (V-(7.7 - 13.0287 \log (Ca_i / Ca_o)))\\
    \end{eqnarray}
2. Sensitivity to $V$: 
    The states $d$ and $f$ do depend on $gCa$, thus we need keep track of their sensitivities. For brevity, we will not write out the derivatives explicitly:
    \begin{eqnarray}
        \frac{\mathrm{d}}{\mathrm{d}t}\frac{\mathrm{d}{Ca}_i}{\mathrm{d}V} &=& 
            \frac{\partial f_{Ca_i}}{\partial Ca_i}\frac{\mathrm{d}{Ca}_i}{\mathrm{d}V} +
            \frac{\partial f_{Ca_i}}{\partial d}\frac{\mathrm{d}d}{\mathrm{d}V} +
            \frac{\partial f_{Ca_i}}{\partial f}\frac{\mathrm{d}f}{\mathrm{d}V} +
            \frac{\partial f_{Ca_i}}{\partial V}
    \end{eqnarray}
3. Sensitivity to $Ca_o$: 
    The states $d$ and $f$ do not depend on $Ca_o$, thus their sensitivities are zero. As a result, the only contribution comes from the rhs of $Ca_i$:
    \begin{eqnarray}
        \frac{\mathrm{d}}{\mathrm{d}t}\frac{\mathrm{d}{Ca}_i}{\mathrm{d}Ca_o} &=& \left( -0.0130287 gCa\, d \, f / Ca_i - 0.07 \right) \frac{\mathrm{d}{Ca}_i}{\mathrm{d}gCa} + 0.0130287 gCa\, d \, f / Ca_o\\
    \end{eqnarray}